In [1]:
import os
import glob
import psycopg2
import pandas as pd
import numpy as np
from SQL_Query import *

In [2]:
try:
    con = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=postgres password=admin")
except psycopg2.Error as e:
    print("Error: Could not make connection to database")
    print(e)

In [3]:
try:
    cur = con.cursor()
except psycopg2.Error as e:
    print("Error: could not get cursor")
    print(e)

In [4]:
def get_data_files(filepath):
    all_files =[]
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for file in files:
            all_files.append(os.path.abspath(file))
    return all_files

### Process song_data

In this first part, you'll perform ETL on the first dataset, song_data, to create the songs and artists dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.

* Use the get_files function provided above to get a list of all song JSON files in data/song_data
* Select the first song in this list
* Read the song file and view the data

In [5]:
song_files = get_data_files("data/song_data")

In [6]:
filepath = song_files[1]

In [7]:
df = pd.read_json(filepath,lines=True)
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,AR8ZCNI1187B9A069B,NaN,NaN,,Planet P Project,SOIAZJW12AB01853F1,Pink World,269.81832,1984


## 1: songs Table
**Extract Data for Songs Table**
<ul>
<li>Select columns for song ID, title, artist ID, year, and duration</li>
<li>Use df.values to select just the values from the dataframe</li>
<li>Index to select the first (only) record in the dataframe</li>
<li>Convert the array to a list and set it to song_data</li>
</ul>

In [8]:
# extract song_data
song_data = df[['song_id','title','artist_id','year','duration']].values[0]
song_data

array(['SOIAZJW12AB01853F1', 'Pink World', 'AR8ZCNI1187B9A069B', 1984,
       269.81832], dtype=object)

**Insert Record into Song Table**

Implement the song_table_insert query in sql_queries.py and run the cell below to insert a record for this song into the songs table. Remember to run create_tables.py before running the cell below to ensure you've created/resetted the songs table in the sparkify database.

In [9]:
try:
    cur.execute(song_table_insert,song_data)
    con.commit()
except psycopg2.Error as e:
    print("Error: Insert record")
    print(e)

## 2: artists Table
**Extract Data for Artists Table**
<ul>
    <li>Select columns for artist ID, name, location, latitude, and longitude</li>
    <li>Use df.values to select just the values from the dataframe</li>
    <li>Index to select the first (only) record in the dataframe</li>
    <li>Convert the array to a list and set it to artist_data</li>
</ul>

In [10]:
artist_data = df[['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']].values[0]
artist_data

array(['AR8ZCNI1187B9A069B', 'Planet P Project', '', nan, nan],
      dtype=object)

In [11]:
try:
    cur.execute(artist_table_insert,artist_data)
    con.commit()
except psycopg2.Error as e:
    print("Error: Insert record")
    print(e)

## Process log_data

In this part, you'll perform ETL on the second dataset, log_data, to create the time and users dimensional tables, as well as the songplays fact table.

Let's perform ETL on a single log file and load a single record into each table.

    1) Use the get_files function provided above to get a list of all log JSON files in data/log_data
    2) Select the first log file in this list
    3) Read the log file and view the data

In [12]:
log_files = get_data_files('data/log_data')

In [13]:
filepath = log_files[0]

In [14]:
df = pd.read_json(filepath,lines=True)
df.head(2)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95 entries, 0 to 94
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   artist         73 non-null     object 
 1   auth           95 non-null     object 
 2   firstName      92 non-null     object 
 3   gender         92 non-null     object 
 4   itemInSession  95 non-null     int64  
 5   lastName       92 non-null     object 
 6   length         73 non-null     float64
 7   level          95 non-null     object 
 8   location       92 non-null     object 
 9   method         95 non-null     object 
 10  page           95 non-null     object 
 11  registration   92 non-null     float64
 12  sessionId      95 non-null     int64  
 13  song           73 non-null     object 
 14  status         95 non-null     int64  
 15  ts             95 non-null     int64  
 16  userAgent      92 non-null     object 
 17  userId         95 non-null     object 
dtypes: float64(2

### 3: time Table
**Extract Data for Time Table**

    1. Filter records by NextSong action
    2. Convert the ts timestamp column to datetime
    3. Extract the timestamp, hour, day, week of year, month, year, and weekday from the ts column and set time_data to a list containing these values in order
    4. Specify labels for these columns and set to column_labels
    5. Create a dataframe, time_df, containing the time data for this file by combining column_labels and time_data into a dictionary and converting this into a dataframe

In [16]:
df = df.query("page == 'NextSong'")

In [17]:
df.head(2)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69


In [18]:
t = pd.to_datetime(df['ts']/1000,unit='s')

In [19]:
t.head()

0   2018-11-11 02:33:56.796000004
1   2018-11-11 02:36:10.796000004
2   2018-11-11 02:40:34.796000004
4   2018-11-11 04:36:13.796000004
5   2018-11-11 04:36:46.796000004
Name: ts, dtype: datetime64[ns]

In [20]:
time_data = np.transpose(np.array([df['ts'].values,t.dt.hour.values,t.dt.day.values,t.dt.week.values,t.dt.month.values,t.dt.year.values,t.dt.weekday.values]))
column_label = ("timestamp", "hour", "day", "week of year", "month", "year", "weekday")

<ipython-input-20-cc623fa22efd>:1: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  time_data = np.transpose(np.array([df['ts'].values,t.dt.hour.values,t.dt.day.values,t.dt.week.values,t.dt.month.values,t.dt.year.values,t.dt.weekday.values]))


In [21]:
time_df = pd.DataFrame(data=time_data,columns=column_label)
time_df.head(1)

,timestamp,hour,day,week of year,month,year,weekday
0,1541903636796,2,11,45,11,2018,6


**Insert Records into Time Table**

Implement the time_table_insert query in sql_queries.py and run the cell below to insert records for the timestamps in this log file into the time table. Remember to run create_tables.py before running the cell below to ensure you've created/resetted the time table in the sparkify database.

In [22]:
for i, row in time_df.iterrows():
    try:
        cur.execute(time_table_insert,list(row))
        con.commit()
    except psycopg2.Error as e:
        print("Error: Insert record")
        print(e)

## 4: users Table

Extract Data for Users Table

    Select columns for user ID, first name, last name, gender and level and set to user_df

In [23]:
user_df = df[["userId", "firstName", "lastName", "gender", "level"]]
user_df.head()

,userId,firstName,lastName,gender,level
0,69,Anabelle,Simpson,F,free
1,69,Anabelle,Simpson,F,free
2,69,Anabelle,Simpson,F,free
4,32,Lily,Burns,F,free
5,75,Joseph,Gutierrez,M,free


In [24]:
for i, row in user_df.iterrows():
    try:
        cur.execute(user_table_insert,row)
        con.commit()
    except psycopg2.Error as e:
        print("Error: Insert record")
        print(e)

## 5: songplays Table
**Extract Data and Songplays Table**

This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the songplays table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.

    1. Implement the song_select query in sql_queries.py to find the song ID and artist ID based on the title, artist name, and duration of a song.
    2. Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to songplay_data

**Insert Records into Songplays Table**

* Implement the songplay_table_insert query and run the cell below to insert records for the songplay actions in this log file into the songplays table. Remember to run create_tables.py before running the cell below to ensure you've created/resetted the songplays table in the sparkify database.

In [26]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (row.ts, row.userId, row.level, songid, artistid, row.sessionId,\
                     row.location, row.userAgent)
    
    cur.execute(songplay_table_insert, songplay_data)
    con.commit()


## Close Connection to Sparkify Database

In [27]:
con.close()